#1. Import thư viện

In [1]:
!pip install pyspark==3.3.1 
!pip install sparknlp
!pip install pyvi
!pip install unidecode
!pip install unicodedata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 26.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=51b4c1a394e04d744643cae30ea3afbf57aec47f672b7934fad1c491a1d2eca9
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.5/469.5 KB 27.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 101.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import re
import unicodedata
import requests
import pandas as pd
import seaborn as sns

from pyvi import ViTokenizer

from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lower, col, regexp_replace, count, mean, when, lit, create_map
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

from pyspark.ml import Pipeline
from pyspark.ml.feature import  Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.linalg import Vectors, VectorUDT

from unidecode import unidecode
from itertools import chain
pd.options.display.max_rows = 9999

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler

#2. Khai báo Spark Session

In [3]:
spark = (
    SparkSession.builder.appName("ModelTraining")
    .config("spark.executor.memory", "16g")
    .getOrCreate()
)

#3. Read DATASET

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###3.1 Read file data.csv (crawl từ web TGDD)

In [6]:
df1_train = spark.read.csv(r'/content/drive/MyDrive/KhóaLuậnTốtNghiệp/BIGDATA/data.csv',header=True, encoding = 'utf-8', inferSchema=True)

In [7]:
df1_train.printSchema()

root
 |-- Name_phone: string (nullable = true)
 |-- Id_comment: string (nullable = true)
 |-- Name_comment: string (nullable = true)
 |-- Buy_place_comment: string (nullable = true)
 |-- Content_comment: string (nullable = true)
 |-- Used_comment: string (nullable = true)



In [8]:
df1_train.show(4)

+-----------------+----------+---------------+-----------------+--------------------+--------------------+
|       Name_phone|Id_comment|   Name_comment|Buy_place_comment|     Content_comment|        Used_comment|
+-----------------+----------+---------------+-----------------+--------------------+--------------------+
|iPhone 14 Pro Max|r-53650660|Nguyễn Tiến Anh|  Đã mua tại TGDD|Dây sạc theo máy ...|Đã dùng khoảng 3 ...|
|iPhone 14 Pro Max|r-53646594|           NHÂN|  Đã mua tại TGDD|Mua cho vợ xài ch...|Đã dùng khoảng 1 ...|
|iPhone 14 Pro Max|r-53642026|           OANH|   Đã mua tại DMX|             Rất tốt|Đã dùng khoảng 3 ...|
|iPhone 14 Pro Max|r-53630446|     Tuyết Ngân|  Đã mua tại TGDD|Sản phẩm rất tốt,...|Đã dùng khoảng 2 ...|
+-----------------+----------+---------------+-----------------+--------------------+--------------------+
only showing top 4 rows



In [9]:
print('Number of rows: \t', df1_train.count())
print('Number of columns: \t', len(df1_train.columns))

Number of rows: 	 3953
Number of columns: 	 6


In [ ]:
#==============================Load data from file=================================
#df = spark.read.csv('/content/drive/MyDrive/KhóaLuậnTốtNghiệp/BIGDATA/data1.csv', header=True, inferSchema=True)

# Loại bỏ các giá trị rỗng trong dataframe
#df = df.dropna()

# Define filter condition
#filter_condition = col("Content_comment") == " "

# Apply filter to DataFrame
#filtered_df = df.filter(filter_condition)
#===================loại bỏ các dòng không cần thiết và giá trị rỗng=======================#
#filtered_df = df.filter((col("Id_comment").isNotNull()) & (col("Id_comment") != "") & (col("Content_comment").isNotNull()) & (col("Content_comment") != ""))

# Save filtered data to file
#filtered_df.write.csv("/content/drive/MyDrive/KhóaLuậnTốtNghiệp/BIGDATA/filtered_file3.csv", header=True)

###3.2 Join 2 dataframe

In [10]:
# data đã loại bỏ các dòng không cần thiết và các dòng chứa giá trị rỗng
df1 = spark.read.csv('/content/drive/MyDrive/KhóaLuậnTốtNghiệp/BIGDATA/filtered_file3.csv', header=True, inferSchema=True)
df2 = spark.read.csv('/content/drive/MyDrive/KhóaLuậnTốtNghiệp/BIGDATA/part-00000-86c6f9eb-dd39-4ae8-9839-cfd7bf5a994f-c000 (1).csv', header=True, inferSchema=True)


In [11]:
print('Number of rows with data after: \t', df1.count())
print('Number of rows with data after: \t',df2.count())

Number of rows with data after: 	 3178
Number of rows with data after: 	 3178


In [12]:
# Chọn cột cần join
df2 = df2.select('Id_comment', 'label')

In [13]:
# Thực hiện join giữa 2 DataFrame
df = df1.join(df2, on='Id_comment', how='inner')

In [14]:
df.show()

+----------+-----------------+---------------+-----------------+--------------------+--------------------+-----+
|Id_comment|       Name_phone|   Name_comment|Buy_place_comment|     Content_comment|        Used_comment|label|
+----------+-----------------+---------------+-----------------+--------------------+--------------------+-----+
|r-53650660|iPhone 14 Pro Max|Nguyễn Tiến Anh|  Đã mua tại TGDD|Dây sạc theo máy ...|Đã dùng khoảng 3 ...|    0|
|r-53646594|iPhone 14 Pro Max|           NHÂN|  Đã mua tại TGDD|Mua cho vợ xài ch...|Đã dùng khoảng 1 ...|    2|
|r-53642026|iPhone 14 Pro Max|           OANH|   Đã mua tại DMX|             Rất tốt|Đã dùng khoảng 3 ...|    1|
|r-53630446|iPhone 14 Pro Max|     Tuyết Ngân|  Đã mua tại TGDD|Sản phẩm rất tốt,...|Đã dùng khoảng 2 ...|    1|
|r-53627015|iPhone 14 Pro Max|   Bùi Thu Hiền|  Đã mua tại TGDD|Em muốn đổi từ ip...|Đã dùng khoảng 2 ...|    2|
|r-53625318|iPhone 14 Pro Max|TRẦN XUÂN NGHĨA|  Đã mua tại TGDD|Rất tốt! Các bạn ...|Đã dùng kho

In [15]:
#xóa những dòng chứa giá trị null
df = df.na.drop()

In [16]:
print('Number of rows with data after: \t',df.count())

Number of rows with data after: 	 3173


In [ ]:
#lưu vài file csv
#df.write.csv("/content/drive/MyDrive/KhóaLuậnTốtNghiệp/BIGDATA/data_comment.csv", header=True)

In [ ]:
#df1 = spark.read.csv('/content/drive/MyDrive/KhóaLuậnTốtNghiệp/BIGDATA/data_comment.csv', header=True, inferSchema=True)

Gán cột label:
*   positive: "1" (những comment đều khen về điện thoại)<br>
*   negative: "0" (những comment không tốt về điện thoại)<br>
*   neutral: "2" (vừa chê vừa khen hoặc những câu không nhằm mục đích đánh giá sản phẩm)


#4. Spark NLP Preprocessing

Tiền xử lý ngôn ngữ tự nhiên (NLP): Các bình luận người dùng được thu thập từ nhiều nguồn khác nhau, do đó đầu tiên cần loại bỏ các dữ liệu không liên quan hoặc không có giá trị, chẳng hạn như các đoạn văn bản quảng cáo, spam hoặc chữ ký. Sau đó, dữ liệu được tiền xử lý NLP bao gồm các bước như tách từ, xử lý từ sai chính tả và đồng nhất hóa từ viết tắt và từ viết hoa.<br>

Tiền xử lý dữ liệu văn bản tiếng Việt: Sau khi được tiền xử lý NLP, dữ liệu cần được xử lý thêm để loại bỏ các từ không có giá trị và chuẩn hóa dữ liệu. Các bước xử lý bao gồm:
<br>
Tách từ: Tiến hành tách từ bằng cách sử dụng các công cụ NLP như VnTokenizer hoặc pyvi để tách từ tiếng Việt.<br>
Loại bỏ stop words: Các stop words (từ dừng) là các từ rất phổ biến và không mang ý nghĩa như "và", "là", "từ", "đi", vv. Do đó, chúng cần được loại bỏ. Chuẩn hóa từ: Tiến hành chuẩn hóa từ bằng cách đưa các từ về dạng gốc (lemma) hoặc dạng đơn (stemming). Điều này giúp loại bỏ sự đa dạng từ vựng của ngôn ngữ tiếng Việt.<br>
Xử lý biểu cảm cảm xúc: Với dữ liệu bình luận, việc phân tích cảm xúc của người dùng là rất quan trọng. Có thể sử dụng các công cụ NLP như VnCoreNLP để phân tích cảm xúc hoặc xây dựng các mô hình máy học để phân loại bình luận theo cảm xúc.<br>
Vector hóa dữ liệu: Cuối cùng, dữ liệu được vector hóa để có thể sử dụng trong các mô hình máy học. Vector hóa dữ liệu là quá trình chuyển đổi dữ liệu văn bản thành các vector số để mô hình có thể xử lý được. Có nhiều phương pháp vector hóa<br>

###Download list stop words Vietnamese and combine and define functions

In [17]:
#Tải danh sách stop words tiếng việt từ github
stop_words_url_1 = 'https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt'
stop_words_data_1 = requests.get(stop_words_url_1).content.decode('utf-8')
stop_words_1 = stop_words_data_1.splitlines()

stop_words_url_2 = 'https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords.txt'
stop_words_data_2 = requests.get(stop_words_url_2).content.decode('utf-8')
stop_words_2 = stop_words_data_2.splitlines()

In [18]:
# kết hợp các danh sách stop words có dấu
stop_words = list(set(stop_words_1 + stop_words_2))

In [19]:
# định nghĩa hàm để loại bỏ các stop words
def remove_stop_words(words):
    return [word for word in words if word not in stop_words]

# định nghĩa hàm để tách câu thành các từ và loại bỏ dấu
def tokenize(text):
    return ViTokenizer.tokenize(text).lower().split()

In [20]:
# đăng ký các hàm với pyspark
remove_stop_words_udf = udf(remove_stop_words, ArrayType(StringType()))
tokenize_udf = udf(tokenize, ArrayType(StringType()))

##4.1 Non-Accent Preprocessing

In [21]:
#lấy 3 cột trong df
df_non_accent = df.select('Id_comment','Content_comment', 'label')
df_non_accent.show()

+----------+--------------------+-----+
|Id_comment|     Content_comment|label|
+----------+--------------------+-----+
|r-53650660|Dây sạc theo máy ...|    0|
|r-53646594|Mua cho vợ xài ch...|    2|
|r-53642026|             Rất tốt|    1|
|r-53630446|Sản phẩm rất tốt,...|    1|
|r-53627015|Em muốn đổi từ ip...|    2|
|r-53625318|Rất tốt! Các bạn ...|    2|
|r-53622496|                  Ok|    1|
|r-53609586|1 sản phẩm đẹp gi...|    1|
|r-53606596|             Rất tốt|    1|
|r-53592098|hàng ngon nhân vi...|    1|
|r-53589531| sản phẩm chất lượng|    1|
|r-53585447|         Hàng OK lắm|    1|
|r-53582199|             Rất tốt|    1|
|r-53581205|Đẹp nhưng 13 prm ...|    0|
|r-53578243|Chất lượng dịch v...|    1|
|r-53577832|         Xài rất tốt|    1|
|r-53573116|nv thế giới dất n...|    2|
|r-53562547|Tgdd gần bến xe r...|    1|
|r-53561310|             Rất tốt|    1|
|r-53560815|                Hihi|    2|
+----------+--------------------+-----+
only showing top 20 rows



In [22]:
# Tạo hàm user-defined function để loại bỏ dấu
remove_diacritic = udf(lambda s: unidecode(s), StringType())

# Loại bỏ dấu trong cột text của DataFrame
df_non_accent = df_non_accent.withColumn("text_without_diacritic", remove_diacritic(col("Content_comment")))

In [23]:
df_non_accent.show()

+----------+--------------------+-----+----------------------+
|Id_comment|     Content_comment|label|text_without_diacritic|
+----------+--------------------+-----+----------------------+
|r-53650660|Dây sạc theo máy ...|    0|  Day sac theo may ...|
|r-53646594|Mua cho vợ xài ch...|    2|  Mua cho vo xai ch...|
|r-53642026|             Rất tốt|    1|               Rat tot|
|r-53630446|Sản phẩm rất tốt,...|    1|  San pham rat tot,...|
|r-53627015|Em muốn đổi từ ip...|    2|  Em muon doi tu ip...|
|r-53625318|Rất tốt! Các bạn ...|    2|  Rat tot! Cac ban ...|
|r-53622496|                  Ok|    1|                    Ok|
|r-53609586|1 sản phẩm đẹp gi...|    1|  1 san pham dep gi...|
|r-53606596|             Rất tốt|    1|               Rat tot|
|r-53592098|hàng ngon nhân vi...|    1|  hang ngon nhan vi...|
|r-53589531| sản phẩm chất lượng|    1|   san pham chat luong|
|r-53585447|         Hàng OK lắm|    1|           Hang OK lam|
|r-53582199|             Rất tốt|    1|               R

In [24]:
#===================loại bỏ các dòng không cần thiết và giá trị rỗng=======================#
df_non_accent = df_non_accent.filter((col("text_without_diacritic").isNotNull()) & (col("text_without_diacritic") != "") & (col("Content_comment").isNotNull()) & (col("Content_comment") != ""))
df_non_accent.show()

+----------+--------------------+-----+----------------------+
|Id_comment|     Content_comment|label|text_without_diacritic|
+----------+--------------------+-----+----------------------+
|r-53650660|Dây sạc theo máy ...|    0|  Day sac theo may ...|
|r-53646594|Mua cho vợ xài ch...|    2|  Mua cho vo xai ch...|
|r-53642026|             Rất tốt|    1|               Rat tot|
|r-53630446|Sản phẩm rất tốt,...|    1|  San pham rat tot,...|
|r-53627015|Em muốn đổi từ ip...|    2|  Em muon doi tu ip...|
|r-53625318|Rất tốt! Các bạn ...|    2|  Rat tot! Cac ban ...|
|r-53622496|                  Ok|    1|                    Ok|
|r-53609586|1 sản phẩm đẹp gi...|    1|  1 san pham dep gi...|
|r-53606596|             Rất tốt|    1|               Rat tot|
|r-53592098|hàng ngon nhân vi...|    1|  hang ngon nhan vi...|
|r-53589531| sản phẩm chất lượng|    1|   san pham chat luong|
|r-53585447|         Hàng OK lắm|    1|           Hang OK lam|
|r-53582199|             Rất tốt|    1|               R

In [25]:
#Tách từ
tokenizer = Tokenizer(inputCol="text_without_diacritic", outputCol="tokenized")

In [26]:
#loại bỏ stopwords trong tiếng việt
stop_words_no_accent = [unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore').decode('utf-8') for word in stop_words]
stop_words_remover = StopWordsRemover(inputCol="tokenized", outputCol="words", stopWords=stop_words_no_accent)

#chuyển đổi các câu văn bản thành các đặc trưng số (features) sử dụng thuật toán hashing trick.
hashing_tf = HashingTF(inputCol="words", outputCol="raw_features")

#đếm số lần xuất hiện của một từ trong tài liệu và tính toán IDF của từ đó trong tập dữ liệu đó. Đánh giá tầm quan trọng
idf = IDF(inputCol="raw_features", outputCol="features", minDocFreq=5)

In [27]:
# Khởi tạo pipeline
pipeline = Pipeline(stages=[tokenizer, stop_words_remover, hashing_tf, idf])

In [28]:
# Áp dụng pipeline lên dataframe
processed_df = pipeline.fit(df_non_accent).transform(df_non_accent)

In [29]:
# Chuẩn hóa cột label thành dạng số
label_indexer = StringIndexer(inputCol="label", outputCol="label_indexed").fit(processed_df)
processed_df = label_indexer.transform(processed_df)

In [30]:
processed_df.show()

+----------+--------------------+-----+----------------------+--------------------+--------------------+--------------------+--------------------+-------------+
|Id_comment|     Content_comment|label|text_without_diacritic|           tokenized|               words|        raw_features|            features|label_indexed|
+----------+--------------------+-----+----------------------+--------------------+--------------------+--------------------+--------------------+-------------+
|r-53650660|Dây sạc theo máy ...|    0|  Day sac theo may ...|[day, sac, theo, ...|[day, sac, may, h...|(262144,[19322,46...|(262144,[19322,46...|          1.0|
|r-53646594|Mua cho vợ xài ch...|    2|  Mua cho vo xai ch...|[mua, cho, vo, xa...|[mua, xai, khog, ...|(262144,[15211,26...|(262144,[15211,26...|          2.0|
|r-53642026|             Rất tốt|    1|               Rat tot|          [rat, tot]|                  []|      (262144,[],[])|      (262144,[],[])|          0.0|
|r-53630446|Sản phẩm rất tốt,...| 

In [31]:
# Chọn các cột cần sử dụng để huấn luyện mô hình
assembler = VectorAssembler(inputCols=["features"], outputCol="featured")
data = assembler.transform(processed_df).select("featured", "label_indexed")

In [32]:
data.printSchema()

root
 |-- featured: vector (nullable = true)
 |-- label_indexed: double (nullable = false)



In [33]:
# Chia dữ liệu thành train_data, dev_data, test_data
train_data, dev_data, test_data = data.randomSplit([0.6, 0.2, 0.2], seed=42)

In [34]:
train_data.printSchema()

root
 |-- featured: vector (nullable = true)
 |-- label_indexed: double (nullable = false)



In [35]:
# Huấn luyện mô hình
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="featured", labelCol="label_indexed")
pipeline = Pipeline(stages=[lr])
model = pipeline.fit(train_data)

In [ ]:
# Đánh giá mô hình trên dev_data
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

predictions = model.transform(dev_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label_indexed", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol="label_indexed", predictionCol="prediction", metricName="f1")
f1 = evaluator.evaluate(predictions)

print("Accuracy_score: {}".format(accuracy))
print("f1_score:: {}".format(f1))

In [ ]:
# Đánh giá mô hình trên test_data
predictions = model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label_indexed", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol="label_indexed", predictionCol="prediction", metricName="f1")
f1 = evaluator.evaluate(predictions)

print("Accuracy: {}".format(accuracy))
print("f1_score:: {}".format(f1))

##Accent Preprocessing

In [ ]:
#lấy 3 cột trong df
df_accent = df.select('Id_comment','Content_comment', 'label')
df_accent.show()

In [ ]:
tokenizer = Tokenizer(inputCol="Content_comment", outputCol="tokenized")

stop_words_no_accent = [unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore').decode('utf-8') for word in stop_words]
stop_words_remover = StopWordsRemover(inputCol="tokenized", outputCol="words", stopWords=stop_words_no_accent)

hashing_tf = HashingTF(inputCol="words", outputCol="raw_features")
idf = IDF(inputCol="raw_features", outputCol="features", minDocFreq=5)

In [ ]:
# Khởi tạo pipeline
pipeline = Pipeline(stages=[tokenizer, stop_words_remover, hashing_tf, idf])

In [ ]:
# Áp dụng pipeline lên dataframe
processed_df = pipeline.fit(df_accent).transform(df_accent)

In [ ]:
# Chuẩn hóa cột label thành dạng số
label_indexer = StringIndexer(inputCol="label", outputCol="label_indexed").fit(processed_df)
processed_df = label_indexer.transform(processed_df)

In [ ]:
processed_df.show()

In [ ]:
# Chọn các cột cần sử dụng để huấn luyện mô hình
assembler = VectorAssembler(inputCols=["features"], outputCol="featured")
data = assembler.transform(processed_df).select("featured", "label_indexed")

In [ ]:
# Chia dữ liệu thành train_data, dev_data, test_data
train_data, dev_data, test_data = data.randomSplit([0.6, 0.2, 0.2], seed=42)

In [ ]:
# Huấn luyện mô hình
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="featured", labelCol="label_indexed")
pipeline = Pipeline(stages=[lr])
model = pipeline.fit(train_data)

In [ ]:
# Đánh giá mô hình trên dev_data
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

predictions = model.transform(dev_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label_indexed", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol="label_indexed", predictionCol="prediction", metricName="f1")
f1 = evaluator.evaluate(predictions)

print("Accuracy_score: {}".format(accuracy))
print("f1_score:: {}".format(f1))

In [ ]:
# Đánh giá mô hình trên test_data
predictions = model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label_indexed", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol="label_indexed", predictionCol="prediction", metricName="f1")
f1 = evaluator.evaluate(predictions)

print("Accuracy: {}".format(accuracy))
print("f1_score:: {}".format(f1))

In [ ]:
#result.write.csv("/content/drive/MyDrive/KhóaLuậnTốtNghiệp/BIGDATA (1)/result.csv", header=True)
#result.toPandas().to_csv("/content/drive/MyDrive/KhóaLuậnTốtNghiệp/BIGDATA (1)/result_accent.csv", index=False)